## Loading dependancies

In [1]:
import cv2
import numpy as np
import glob
import os
import inflect
import pandas as pd
import uuid
from moviepy.editor import *
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import wave
import scipy.io.wavfile
import matplotlib.pyplot as plt
import random
from IPython.display import Audio
import seaborn as sns
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import tqdm

## Part 1: Creating video frames from .mp4 files

Load data i.e TikTok emoji challenge 

In [2]:
# All videos are saved here!
main_directory = "E:\\tiktok"

skip_files = ["raw",
              "ready",
              "try to make a face that fits my description",
              "face challenge",
              "clown face"]


all_emo_files=[main_directory+"\\"+ i for i in os.listdir(main_directory) if i not in skip_files]

The follownig faces are prompted in the TikTok emoji challenge

In [3]:
all_emo_files

['E:\\tiktok\\angry face',
 'E:\\tiktok\\crying face',
 'E:\\tiktok\\disappointed face',
 'E:\\tiktok\\Extracted Faces',
 'E:\\tiktok\\grinning face with clenched teeth',
 'E:\\tiktok\\grinning face with normal eyes',
 'E:\\tiktok\\nauseated face',
 'E:\\tiktok\\slightly smiling face',
 'E:\\tiktok\\thinking face',
 'E:\\tiktok\\winking face']

Extracting video frames and creating a seperate directory for each emotion

In [4]:
path_frames = os.path.join(main_directory,"Extracted Faces"); os.mkdir(path_frames)

for emo in all_emo_files:
    print("Reading ->",emo)
    list_of_vids = []
    
    for paths in glob.glob(os.path.join(emo, "*mp4")):
        list_of_vids.append(paths)
    path_frames_emotion = os.path.join(path_frames,emo[len(main_directory)+1:])
    os.mkdir(path_frames_emotion)
    
    for v in tqdm.tqdm(list_of_vids):
        vidcap = cv2.VideoCapture(v)
        success,image = vidcap.read()
        count = 0
        
        while success:
            cv2.imwrite(path_frames_emotion+"\\"+"{}_frame_{}.jpg".format(v[len(emo)+1:][:-4],count+1), image)
            success,image = vidcap.read()
            count += 1

## Part 2: Detecting and scaling facial expressions + Eliminating dim lit frames

In [5]:
## Creating a sub-directory
faces_normalized = os.path.join("E:\\tiktok\\Extracted Faces","faces normalized")
os.mkdir(faces_normalized)

In [6]:
path_all_faces = "E:\\tiktok\\Extracted Faces"

for i in tqdm.tqdm(range(len(os.listdir(path_all_faces)))):
    expression = os.path.join(faces_normalized,os.listdir(path_all_faces)[i])
    os.mkdir(expression)
        # Initialize face counter
    face_count = 1
    list_of_imgs = os.path.join(path_all_faces,os.listdir(path_all_faces)[i])
    
    for img in (glob.glob(os.path.join(list_of_imgs,"*jpg"))):

        # Read image using cv2.imread()
        image = cv2.imread(img)

        # Construct HAAR cascade 
        faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

        # Locate all face coordinates
        faces = faceCascade.detectMultiScale(image, scaleFactor=1.5, minNeighbors=10, minSize=(50, 50))

        # If you have rotated images
        timeout = 3  # Set a timer to exit an infinite while loop if it occurs
        
        while (len(faces)==0 and timeout!=0):
            # Iteratively rotate images clockwise to detect faces
            image = cv2.rotate(image,cv2.ROTATE_90_CLOCKWISE)

            # Construct HAAR cascade
            faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

            # Locate all face coordinates
            faces = faceCascade.detectMultiScale(image, scaleFactor=2.5, minNeighbors=11, minSize=(50, 50))

            # Countdown
            timeout-=1

        # Read faces 
        for x,y,h,w in faces:
            cv2.imwrite(expression+"\\"+"{}_face {}.jpg".format(img[len(list_of_imgs)+1:][:-4],
                                                                face_count),image[y:y+h, x:x+w])
            face_count+=1